<a href="https://colab.research.google.com/github/hongsy0113/cose461-NLP-project-choi-hong/blob/main/sns_text_classification_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import json
from pandas import json_normalize


In [2]:
# json data 파일들을 google drive에 저장
# google drive 에 있는 파일들을 접근하기 위해 mount
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
# load json file as below
# 참고로 행사3.json 파일은 디버그 용으로 dialogue 두개만 있는 파일임.
with open('/content/gdrive/MyDrive/cose461/행사3.json', 'r') as f:

    jdata = json.load(f)

FileNotFoundError: ignored

In [4]:
# Convert json file to dataFrame
# input file parameter should be str, path and name of json file
# output dataFrame contains utterances. 
# each utterance has dialogueInfo and participantsInfo (i.e gender)

def json2df (file):
  with open(file, 'r') as f:
    jdata = json.load(f)
  
  meta_list = [['header', 'dialogueInfo', 'numberOfParticipants'],
             ['header', 'dialogueInfo', 'numberOfUtterances'],
             ['header', 'dialogueInfo', 'numberOfTurns'],
             ['header', 'dialogueInfo', 'type'],
             ['header', 'dialogueInfo', 'topic'],
             ['header', 'dialogueInfo', 'dialogueID'],
            ]
  # dataframe of utterances 
  df1 = pd.json_normalize(jdata['data'][:], record_path=['body'], meta= meta_list, errors='ignore')
  # dataframe of participants info
  df2 = pd.json_normalize(jdata['data'], record_path=['header', 'participantsInfo'],  meta= [['header', 'dialogueInfo', 'dialogueID']], errors='ignore')
  # join df1 and df2 where (df1.dialogueID == df2.dialogueID and df1.participantID == df2.participantID)
  df = pd.merge(left = df1 , right = df2, how = "left", on = ["header.dialogueInfo.dialogueID", 'participantID' ],sort=False)

  # rename column names
  # 열이름 너무 길어서 바꿈. P는 참가자, U는 메시지, D는 대화, T는 turn 으로 통일시킴. 맘에 안 들면 바꿔도 ok
  df.rename(columns = {'header.dialogueInfo.numberOfParticipants' : 'P_num', 
                       'header.dialogueInfo.numberOfUtterances': 'U_num', 
                       'header.dialogueInfo.numberOfTurns':'T_num',
                       'header.dialogueInfo.type': 'D_type',
                       'header.dialogueInfo.topic': 'D_topic',
                       'header.dialogueInfo.dialogueID':'D_id',
                       'utteranceID':'U_id',
                       'participantID':'P_id',
                       'turnID':'T_id',
                       'date':'U_date',
                       'gender':'P_gender',
                       'residentialProvince':'P_residential',
                       'age':'P_age'}, inplace = True)
  # 대화 id, utterance id 를 index로 함. 마찬가지로 수정해도 됨.
  df.set_index(['D_id','U_id'], inplace=True)
  return df


In [5]:
# 행사.json, 식음료.json 등 여러 데이터셋을 하나로 만들기 위한 함수
def concat_json2df (file_list):
  result = None

  for i, file in enumerate(file_list):
    df = json2df(file)
    if (i==0):
      result = df.copy()
    else:
      result = pd.concat([result,df])
  return result

In [6]:
f_list = ['/content/gdrive/MyDrive/cose461/행사.json',
          '/content/gdrive/MyDrive/cose461/식음료.json']

test3 = concat_json2df(f_list)

test3.describe()

,utterance,P_id,U_date,T_id,time,P_num,U_num,T_num,D_type,D_topic,P_age,P_residential,P_gender
count,3381752,3381752,3381752,3381752,3381752,3381752,3381752,3381752,3381752,3381752,3381752,3381752,3381752
unique,2543121,9,2873,20,1440,8,23,17,2,2,9,18,2
top,#@시스템#사진#,P01,2020-12-21,T3,17:55:00,2,20,8,일상 대화,식음료,20대,서울특별시,여성
freq,38365,1759889,8714,455462,5287,2981713,499120,599571,3380905,1821031,2470129,969946,2713080


In [7]:
# example of json2df function
x = json2df('/content/gdrive/MyDrive/cose461/행사3.json')
x

FileNotFoundError: ignored

In [9]:
# example of json2df function
test2 = json2df('/content/gdrive/MyDrive/cose461/행사.json')


In [8]:
test2.describe()

NameError: ignored

In [10]:
meta_list = [['header', 'dialogueInfo', 'numberOfParticipants'],
             ['header', 'dialogueInfo', 'numberOfUtterances'],
             ['header', 'dialogueInfo', 'numberOfTurns'],
             ['header', 'dialogueInfo', 'type'],
             ['header', 'dialogueInfo', 'topic'],
             ['header', 'dialogueInfo', 'dialogueID'],
             
            ]
             #['header', 'participantsInfo', 'residentialProvince'],
             #['header', 'participantsInfo', 'gender'],
             #['header', 'participantsInfo', 'participantID']]
df1 = pd.json_normalize(jdata['data'][:], record_path=['body'], meta= meta_list, errors='ignore')
df2 = pd.json_normalize(jdata['data'], record_path=['header', 'participantsInfo'],  meta= [['header', 'dialogueInfo', 'dialogueID']], errors='ignore')
df2

pd.merge(left = df1 , right = df2, how = "left", on = ["header.dialogueInfo.dialogueID", 'participantID' ],sort=False)


NameError: ignored

In [ ]:
#test test test
# 이미 구현되어 있는 형태소 분석 툴을 써야 할 텐데 
# '괜찮을거가태' 처럼 표준형태와 다른 형태의 단어들이 우리 모델 학습에 중요할 거 같은데
# 이런 거 어케할지. 그냥 전처리 없이 raw단어로 할지

In [28]:
# Naive-Bayes Method.
test = test3[['utterance', 'P_gender']]
test['P_gender'].value_counts()

여성    2713080
남성     668672
Name: P_gender, dtype: int64

In [39]:
# Gender to discrete variable
# 0 : '여성', 1 : '남성'
test['P_gender'] = [0 if gender=='여성' else 1 for gender in test['P_gender']]
test

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


utterance  P_gender
D_id                                 U_id                                      
223740cf-606e-5149-820c-7dce69513da1 U1                     8월 말도 ㄱㅊ해         0
                                     U2                  그때도 글ㅋㅔ 안추울걸         0
                                     U3                      29일로 할까?         0
                                     U4    날짜라도 일단 확정하면 ㅋㅋㅋ 찾기 수월하니까~         0
                                     U5                  응응 그럼 29일로가자         0
...                                                               ...       ...
fd4c483f-28a3-5703-82d3-2e3b1a984078 U12                         리조또랑         0
                                     U13                            웅         0
                                     U14                          수육에         0
                                     U15                           헤헤         0
                                     U16                         장난아냐         0

[3381752 rows x 2 columns]

In [67]:
test_women = test[test['P_gender']==0]
test_men = test[test['P_gender']==1]

print("Number of women : {}, Number of men : {}".format(test_women['P_gender'].value_counts()[0], test_men['P_gender'].value_counts()))

Number of women : 2713080, Number of men : 1    668672
Name: P_gender, dtype: int64


In [84]:
test_women = test_women.sample(n=668672, random_state=7)
test = pd.concat([test_women, test_men], sort=True)
test

P_gender       utterance
D_id                                 U_id                          
4b84c886-16af-570f-ae9c-467e2c7c16fd U17          0       #@시스템#사진#
2c903f6f-1cd3-56d3-aa7b-c20f2d5bebe4 U13          0            샐러드?
d7a1f855-9b3a-5ade-b171-007d90ff1961 U10          0       둘중에 하나 골라
a92da65a-927b-51d1-a004-d23abf2ad3c0 U6           0  찾아보는데 메뉴 뭔데 진짜
774ff6d4-1622-5056-841d-7ed37be148b9 U5           0       푸딩으로 입가심할
...                                             ...             ...
68161a7a-77e5-58b2-8861-ca4d27e73019 U7           1          예를들어??
                                     U8           1        블랑 이런거??
                                     U11          1              옹옹
                                     U15          1         앟 ㅏ을게쏭ㅇ
                                     U16          1       저런걸로 사놓을겡

[1337344 rows x 2 columns]

In [91]:
X = test['utterance']
y = test['P_gender']

from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=7)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=7)

vectorizer = CountVectorizer(min_df=10).fit(X_train)
X_train_countvector = vectorizer.transform(X_train)

feature_names = vectorizer.get_feature_names()
print(feature_names)

['00', '000원', '09', '0칼로리', '10', '100', '100개', '10개', '10만원', '10만원은', '10번', '10분', '10분까지', '10분뒤에', '10분만', '10분에', '10분정도', '10분쯤', '10시', '10시까지', '10시반', '10시반에', '10시부터', '10시에', '10시쯤', '10월', '10월에', '10일', '10일날', '10일에', '10일은', '10키로', '11', '11번', '11시', '11시까지', '11시나', '11시반', '11시반까지', '11시반에', '11시부터', '11시에', '11시쯤', '11시쯤에', '11월', '11월에', '11일', '11일에', '12', '12시', '12시간', '12시까지', '12시나', '12시로', '12시면', '12시반', '12시반에', '12시반쯤', '12시부터', '12시에', '12시쯤', '12월', '12월에', '12일', '12일에', '13', '13분', '13일', '13일날', '13일에', '14', '14일', '14일날', '14일에', '15', '15만원', '15분', '15분에', '15분정도', '15분쯤', '15일', '15일에', '16', '16일', '16일에', '17', '17일', '17일에', '18', '18일', '18일에', '19', '19일', '19일에', '1개', '1개씩', '1년', '1년에', '1도', '1등', '1리터', '1명', '1박', '1박2일', '1박2일로', '1박에', '1박하고', '1번', '1병', '1분', '1시', '1시간', '1시간정도', '1시까지', '1시로', '1시반', '1시부터', '1시에', '1시쯤', '1월', '1월에', '1위', '1인', '1인당', '1인분', '1인분에', '1일', '1일에', '1차', '1층', '1층에', '1키로에', '1호선', '1회', '20

In [89]:
classifier = MultinomialNB()
classifier.fit(X_train_countvector, y_train)

y_predict = classifier.predict(vectorizer.transform(X_val))
cm = confusion_matrix(y_val, y_predict)
df_cm = pd.DataFrame(cm, index = ['실제값(0)', '실제값(1)'], columns = ['예측값(0)', '예측값(1)'])
print(classification_report(y_val, y_predict))


              precision    recall  f1-score   support

           0       0.52      0.80      0.63    133504
           1       0.57      0.26      0.36    133965

    accuracy                           0.53    267469
   macro avg       0.54      0.53      0.49    267469
weighted avg       0.54      0.53      0.49    267469



In [90]:
print(df_cm)

        예측값(0)  예측값(1)
실제값(0)  107148   26356
실제값(1)   99129   34836


In [50]:
y_test_predict = classifier.predict(vectorizer.transform(X_test))
print(classification_report(y_test, y_test_predict))

              precision    recall  f1-score   support

           0       0.81      0.99      0.89    542377
           1       0.51      0.05      0.09    133974

    accuracy                           0.80    676351
   macro avg       0.66      0.52      0.49    676351
weighted avg       0.75      0.80      0.73    676351

